In [ ]:
#Code to create panels a,b Figure 1 in Jong etc al. submitted publication

In [1]:
import numpy as np
import math
import xarray as xr
from numpy import ma

import sys
import os
import traceback
import subprocess
import tempfile
import eofs

import matplotlib.pyplot as plt
import matplotlib.font_manager
import matplotlib.patches as mpatches

matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
import cartopy
import cartopy.crs as ccrs


import datetime
today=datetime.date.today()


import matplotlib.ticker as ticker
plt.rcParams['font.family'] = 'Arial'
#plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'

In [ ]:
# All these files assume that the notebooks LD_SST_multivariate_regressions.ipynb and LD_SST_multivariate_regressions_smoothed.ipynb have been run successfull and outputs saved

# unsmoothed all proxies
corr_coeff = xr.open_dataset('outputs/unsmoothed/corr_coeff.nc')
conf_int = xr.open_dataset('outputs/unsmoothed/ssta_cis.nc')
ssta_pred = xr.open_dataset('outputs/unsmoothed/ssta_pred.nc')
ssta_reco = xr.open_dataset('outputs/unsmoothed/ssta_reco.nc')

#unsmoothed only summer sea salts
corr_coeff_sss = xr.open_dataset('outputs/unsmoothed/corr_coeff_sss.nc')
conf_int_sss = xr.open_dataset('outputs/unsmoothed/ssta_sss_cis.nc')
ssta_pred_sss = xr.open_dataset('outputs/unsmoothed/ssta_pred_sss.nc')
ssta_reco_sss = xr.open_dataset('outputs/unsmoothed/ssta_reco_sss.nc')

corr_coeff_sss_wss = xr.open_dataset('outputs/unsmoothed/corr_coeff_sss_wss.nc')
conf_int_sss_wss = xr.open_dataset('outputs/unsmoothed/ssta_sss_wss_cis.nc')
ssta_pred_sss_wss = xr.open_dataset('outputs/unsmoothed/ssta_pred_sss_wss.nc')
ssta_reco_sss_wss = xr.open_dataset('outputs/unsmoothed/ssta_reco_sss_wss.nc')

#smoothed files all species
corr_coeff_smooth = xr.open_dataset('outputs/smoothed/corr_coeff_smoothed.nc')
conf_int_smooth = xr.open_dataset('outputs/smoothed/ssta_cis_smoothed.nc')
ssta_reco_smooth = xr.open_dataset('outputs/smoothed/ssta_reco_smoothed.nc')
ssta_pred_smooth = xr.open_dataset('outputs/smoothed/ssta_pred_smoothed.nc')

#only summer sea salts
corr_coeff_sss_smooth = xr.open_dataset('outputs/smoothed/corr_coeff_sss_smoothed.nc')
conf_int_sss_smooth = xr.open_dataset('outputs/smoothed/ssta_sss_cis_smoothed.nc')
ssta_pred_sss_smooth = xr.open_dataset('outputs/smoothed/ssta_pred_sss_smoothed.nc')
ssta_reco_sss_smooth = xr.open_dataset('outputs/smoothed/ssta_reco_sss_smoothed.nc')

#winter and summer sea salts
corr_coeff_sss_wss_smooth = xr.open_dataset('outputs/smoothed/corr_coeff_sss_wss_smoothed.nc')
conf_int_sss_wss_smooth=xr.open_dataset('outputs/smoothed/ssta_sss_wss_cis_smoothed.nc')
ssta_pred_sss_wss_smooth = xr.open_dataset('outputs/smoothed/ssta_pred_sss_wss_smoothed.nc')
ssta_reco_sss_wss_smooth = xr.open_datasset('outputs/smoothed/ssta_reco_sss_wss_smoothed.nc')

In [ ]:
cis_mask=conf_int['conf_interval'][0]*conf_int['conf_interval'][1]
sig_mask=np.where(cis_mask<0.0,0.0,1.0)

cis_mask_sss=conf_int_sss['conf_interval'][0]*conf_int_sss['conf_interval'][1]
sig_mask_sss=np.where(cis_mask_sss<0.0,0.0,1.0)

cis_mask_sss_wss=conf_int_sss_wss['conf_interval'][0]*conf_int_sss_wss['conf_interval'][1]
sig_mask_sss_wss=np.where(cis_mask_sss_wss<0.0,0.0,1.0)

#smoothed
cis_mask_smooth=conf_int_smooth['conf_interval'][0]*conf_int_smooth['conf_interval'][1]
sig_mask_smooth=np.where(cis_mask_smooth<0.0,0.0,1.0)

cis_mask_sss_smooth=conf_int_sss_smooth['conf_interval'][0]*conf_int_sss_smooth['conf_interval'][1]
sig_mask_sss_smooth=np.where(cis_mask_sss_smooth<0.0,0.0,1.0)

In [ ]:
names=["DSSmain", "DSS97", "DSS99", "DSS1617"]
lats=[-66.769722,-66.777220,-66.770555,-66.773917]
lons=[112.806944,112.811388,112.806944,112.811617]

labelfontsize=12
textfontsize=12
plt.rcParams.update({'hatch.color': 'k'})


mask_diff1=abs(sig_mask-sig_mask_sss)

mask_diff2=abs(sig_mask_smooth-sig_mask) #is absolute value the diff

import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LongitudeFormatter, LatitudeFormatter,LatitudeLocator

levels=np.linspace(0,0.6,7)
conlevels=np.linspace(0,1,2)
print(conlevels,levels)
cm = 1/2.54

proj = ccrs.PlateCarree(central_longitude=180)
land=cartopy.feature.NaturalEarthFeature('physical', 'land', \
    scale='50m', edgecolor='k',facecolor='none')
fig, ax = plt.subplots(2, 1, figsize=(8,9), sharex=True,gridspec_kw={'hspace': 0.02},subplot_kw=dict(projection=proj))

# Full linear regression correlation coefficient, and confidence interval derived significance.
# hatching from the different in significance between 3-var relationship and just the sea salts (using confidence intervals to create masks)

cs0 = ax[0].pcolormesh(corr_coeff.lon.values, corr_coeff.lat.values, corr_coeff['corr_coefficient'], transform=ccrs.PlateCarree(),cmap='Purples',vmin=0,vmax=0.6,shading='gouraud')
sigcontour20=ax[0].contour(corr_coeff.lon.values, corr_coeff.lat.values,sig_mask,transform=ccrs.PlateCarree(),colors='k',levels=[0.95],linewidths=0.5)
sigcontour30=ax[0].contour(corr_coeff.lon.values, corr_coeff.lat.values,mask_diff1,transform=ccrs.PlateCarree(),colors='k',levels=[0.95],linewidths=0.5)
hatch0=ax[0].contourf(corr_coeff.lon.values, corr_coeff.lat.values,mask_diff1, transform=ccrs.PlateCarree(),colors='none',levels=[0.99,1],hatches=['..','None'])
shade0=ax[0].contourf(corr_coeff.lon.values, corr_coeff.lat.values,sig_mask, transform=ccrs.PlateCarree(),levels=(0,0.95),cmap='Greys_r',alpha=0.3)
hatch0._hatch_color = (0,0,0,1)

ax[0].plot(lons[0], lats[0], linewidth=2, markersize='10', marker='*',color='black',transform=ccrs.Geodetic()) #mark law dome
ax[0].add_feature(land,linewidth=0.5);ax[0].set_global()
#ax[0].clabel(sigcontour20, fontsize=10)

#double check the masking for the difference in the significance.
#cs1 = ax[1].contourf(ersst.lon.values, ersst.lat.values, corr_coeff_smooth['corr_coefficient'], transform=ccrs.PlateCarree(),cmap='Purples',levels=levels,extend='both')
cs1 = ax[1].pcolormesh(corr_coeff_smooth.lon.values, corr_coeff_smooth.lat.values, corr_coeff_smooth['corr_coefficient'], transform=ccrs.PlateCarree(),cmap='Purples',vmin=0,vmax=0.6,shading='gouraud')
sigcontour2=ax[1].contour(corr_coeff_smooth.lon.values, corr_coeff_smooth.lat.values,sig_mask_smooth,transform=ccrs.PlateCarree(),colors='k',levels=[0.95],linewidths=0.5)
sigcontour3=ax[1].contour(corr_coeff_smooth.lon.values, corr_coeff_smooth.lat.values,mask_diff2,transform=ccrs.PlateCarree(),colors='k',levels=[0.95],linewidths=0.5)

hatch=ax[1].contourf(corr_coeff_smooth.lon.values, corr_coeff_smooth.lat.values,mask_diff2, transform=ccrs.PlateCarree(),colors='none',levels=[0.99,1], hatches=['///',None])
hatch._hatch_color = (0, 0, 0, 1.0)
shade=ax[1].contourf(corr_coeff_smooth.lon.values, corr_coeff_smooth.lat.values,sig_mask_smooth, transform=ccrs.PlateCarree(),levels=[0,0.95],cmap='Greys',alpha=0.3)
ld,=ax[1].plot(lons[0], lats[0], linewidth=0, markersize='10', marker='*',color='black',transform=ccrs.Geodetic(),label='Law Dome')

ax[1].add_feature(land,linewidth=0.5);ax[1].set_global()



cax = ax[1].inset_axes([0.01, -0.15, 0.65, 0.04],transform = ax[1].transAxes)

cb=fig.colorbar(cs1,cax=cax,orientation='horizontal',extend='max',pad=0.2,label='Correlation Coefficient')
cb.set_label(label='Correlation Coefficient',size=labelfontsize)

cb.ax.tick_params(labelsize=labelfontsize)

gl0 = ax[0].gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='gray', alpha=0.5 )
gl0.left_labels = True
gl0.top_labels = True
gl0.bottom_labels = False
gl0.xlocator = mticker.FixedLocator([0, 60, 120, 180, -120, -60])
gl0.ylocator = LatitudeLocator()
gl0.xformatter = LongitudeFormatter()
gl0.yformatter = LatitudeFormatter()
gl0.xlabel_style = {'size': labelfontsize}
gl0.ylabel_style = {'size': labelfontsize}


gl = ax[1].gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1, color='gray', alpha=0.5 )
gl.left_labels = True
gl.top_labels = False
gl.bottom_labels = True
gl.xlocator = mticker.FixedLocator([0, 60, 120, 180, -120, -60])
gl.ylocator = LatitudeLocator()
gl.xformatter = LongitudeFormatter()
gl.yformatter = LatitudeFormatter()
gl.xlabel_style = {'size': labelfontsize}
gl.ylabel_style = {'size': labelfontsize}



shade_patch = mpatches.Patch(color='grey', alpha=0.5,label='Not significant')
hatch_patch = mpatches.Patch(facecolor='indigo', alpha=0.5, edgecolor='black',hatch='///',label='$\Delta$ significance vs unsmoothed')
stip_patch = mpatches.Patch(facecolor='indigo', alpha=0.5, edgecolor='black',hatch='..',label='Not significant with only SSS')

ax[1].legend(handles=[ld,shade_patch,stip_patch,hatch_patch],bbox_to_anchor=(0.99,-0.32),  loc="lower right", bbox_transform=ax[1].transAxes,fontsize=8)
levels=np.linspace(0,0.6,7)

plt.text(-0.05, 1, '(a)',
     size=textfontsize,fontweight='bold',
     horizontalalignment='center',
     verticalalignment='center',
     transform = ax[0].transAxes)

plt.text(-0.05, 1, '(b)',
     size=textfontsize,fontweight='bold',
     horizontalalignment='center',
     verticalalignment='center',
     transform = ax[1].transAxes)

plt.savefig('plots/Fig1.png',bbox_inches='tight')